In [1]:
import sys
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')
sys.path.insert(2, '/Users/mac/Desktop/PycharmProjects/TAADL/models')

In [2]:
import logging
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

from config import DATA_PATH
from typing import Tuple
from network import GPCopulaNet
from torch.distributions import MultivariateNormal
from utils import train_test_split, loss_GLL, transform, inv_transform_3D
from loader import TrainDataset

# Data Preprocessing

In [5]:
# load and preprocess the data
feature = pd.read_csv(DATA_PATH+'log_return.csv', index_col='Date')
df_tr, df_te = train_test_split(feature)

labels = df_tr.shift(-1)
df_tr, labels = df_tr.iloc[:-1], labels.iloc[:-1]

inputs, labels = torch.Tensor(df_tr.values), torch.Tensor(labels.values)
indice = torch.randperm(7)[:3]

inputs_x, cdf = transform(inputs[:500])

In [6]:
net = GPCopulaNet(input_dim=1, hidden_dim=15, embed_dim=6, num_layers=2,
                num_assets=7, seq_length=24, pred_length=4, batch_size=1, rank_size=2)

net.init_weight()
optimizer = torch.optim.Adam(params=net.parameters(), lr=1e-3)

In [7]:
mu,cov = net(inputs[:24], indice)
loss = loss_GLL(inputs_x[:24][:,indice], mu, cov)

optimizer.zero_grad()
loss.backward()
optimizer.step()

loss, mu.size(), cov.size()

(tensor(84.1235, grad_fn=<NegBackward0>),
 torch.Size([24, 3, 1]),
 torch.Size([24, 3, 3]))

In [14]:
x_hat = net.predict(inputs[24:25],20)
z_hat = inv_transform_3D(x_hat, cdf)

In [20]:
z_mean, z_std = z_hat.mean(axis=0), z_hat.std(axis=0)

In [25]:
inputs[24:28]

tensor([[ 0.0197,  0.0266, -0.0013, -0.0050, -0.0069, -0.0057,  0.0138],
        [ 0.0029,  0.0168, -0.0033, -0.0011,  0.0135,  0.0097,  0.0038],
        [ 0.0094,  0.0141,  0.0003, -0.0041,  0.0240,  0.0096,  0.0021],
        [ 0.0189,  0.0039, -0.0054, -0.0065, -0.0483,  0.0176, -0.0064]])

In [21]:

z_mean

tensor([[ 0.0036,  0.0075,  0.0017,  0.0017, -0.0068,  0.0031,  0.0018],
        [ 0.0102,  0.0151,  0.0021, -0.0013,  0.0033,  0.0128, -0.0023],
        [-0.0008,  0.0107,  0.0021, -0.0016, -0.0049,  0.0030,  0.0050],
        [ 0.0028,  0.0074,  0.0013,  0.0005,  0.0116, -0.0076,  0.0010]])

In [22]:
z_std

tensor([[0.0256, 0.0306, 0.0032, 0.0061, 0.0283, 0.0268, 0.0154],
        [0.0262, 0.0366, 0.0050, 0.0067, 0.0363, 0.0243, 0.0256],
        [0.0280, 0.0234, 0.0024, 0.0081, 0.0265, 0.0216, 0.0187],
        [0.0184, 0.0404, 0.0047, 0.0052, 0.0204, 0.0319, 0.0154]])